In [ ]:
from importlib import import_module
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Path to your src folder
SRC_DIR = Path("../src").resolve()
sys.path.append(str(SRC_DIR))

# Import your modules
benchmark_mod = import_module("benchmark")
mom12_0_mod = import_module("momentum_12_0")
mom12_1_mod = import_module("momentum_12_1")

# Extract the dataframes
benchmark_df = benchmark_mod.benchmark_df
mom12_0_df = mom12_0_mod.mom12_0_df
mom12_1_df = mom12_1_mod.mom12_1_df
prev_perf_12_0 = mom12_0_mod.prev_perf_12_0
prev_perf_12_1 = mom12_1_mod.prev_perf_12_1
vol_perf_12_0 = mom12_0_mod.vol_perf_12_0
vol_perf_12_1 = mom12_1_mod.vol_perf_12_1

In [ ]:
# Save cumulative growth plot as PDF
def plot_cumulative_growth(benchmark_df, mom12_1_df, mom12_0_df, save_path="figure1.pdf"):
    plt.figure(figsize=(12, 12))

    for df, label, style, color in zip(
        [benchmark_df, mom12_1_df, mom12_0_df],
        ["Benchmark", "12-1 Momentum", "12-0 Momentum"],
        ["-", "-", "--"],
        ["grey", "black", "black"]
    ):
        cumval = df["cumvalue"]
        total_return = cumval.iloc[-1] - 1
        cagr = cumval.iloc[-1]**(1/(len(cumval)/12)) - 1
        plt.plot(df.index, cumval, style, color=color, label=f'{label} ($1 → ${cumval.iloc[-1]:.2f})')
        plt.text(df.index[-1], cumval.iloc[-1], f'Total Return: {total_return*100:.2f}%\nCAGR: {cagr*100:.2f}%', 
                 ha='left', va='top', fontsize=10, color=color)

    plt.title('Cumulative Growth of $1 Invested in Each Portfolio')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Value ($)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    
    plt.savefig(save_path, format='pdf')  # save as PDF
    plt.show()

# Call the function
plot_cumulative_growth(benchmark_df, mom12_1_df, mom12_0_df)


In [ ]:
# Map regime names to short labels for x-axis
regime_short = {"Below Average Previous Return Month": "Below Avg",
                "Above Average Previous Return Month": "Above Avg"}

def plot_return_regime_bar_grouped_clean(df_12_1, df_12_0, regime_col, title, save_as=None):
    # Compute mean returns
    mean_12_1 = [df_12_1.loc[df_12_1[regime_col] == r, "Long_Return"].mean()*100
                 for r in ["Below Average Previous Return Month", "Above Average Previous Return Month"]]
    mean_12_0 = [df_12_0.loc[df_12_0[regime_col] == r, "Long_Return"].mean()*100
                 for r in ["Below Average Previous Return Month", "Above Average Previous Return Month"]]

    # Combine into one list: 12-1 first, then 12-0
    mean_all = mean_12_1 + mean_12_0

    # X-axis labels: just regime names, strategy distinguished by color/hatch
    labels = ["Below Avg", "Above Avg", "Below Avg", "Above Avg"]

    x = np.arange(len(labels))  # positions for 4 bars
    fig, ax = plt.subplots(figsize=(12,6))

    # Plot bars
    ax.bar(x[:2], mean_all[:2], color='black', label='12-1')  # first 2 bars solid black
    ax.bar(x[2:], mean_all[2:], color='white', edgecolor='black', hatch='//', label='12-0')  # last 2 bars hatched

    ax.set_xticks(x)
    ax.set_xticklabels(labels, fontsize=12, rotation=0)
    ax.set_ylabel("Mean Monthly Return (%)", fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    ax.legend(loc='upper right', fontsize=12)

    plt.tight_layout()
    if save_as:
        plt.savefig(save_as, format='pdf')
    plt.show()

# Call the function
plot_return_regime_bar_grouped_clean(
    prev_perf_12_1,
    prev_perf_12_0,
    "PrevMonth_Regime",
    "Mean Monthly Returns by Previous Month Return Regime",
    save_as="figure2.pdf"
)


In [ ]:
# Short labels for x-axis
vol_label_map = {
    "Below Average Volatility": "Below Avg",
    "Above Average Volatility": "Above Avg"
}

def plot_volatility_regime_bar_grouped(df_12_1, df_12_0, regime_col, title, save_as=None):
    # Compute mean returns
    mean_12_1 = [df_12_1.loc[df_12_1[regime_col] == r, "Long_Return"].mean()*100
                 for r in ["Below Average Volatility", "Above Average Volatility"]]
    mean_12_0 = [df_12_0.loc[df_12_0[regime_col] == r, "Long_Return"].mean()*100
                 for r in ["Below Average Volatility", "Above Average Volatility"]]

    # Combine into one list: 12-1 first, then 12-0
    mean_all = mean_12_1 + mean_12_0

    # X-axis labels
    labels = ["Below Avg", "Above Avg", "Below Avg", "Above Avg"]

    x = np.arange(len(labels))
    fig, ax = plt.subplots(figsize=(12,6))

    # Plot bars
    bars_12_1 = ax.bar(x[:2], mean_all[:2], color='black', label='12-1')  # 12-1 bars
    bars_12_0 = ax.bar(x[2:], mean_all[2:], color='white', edgecolor='black', hatch='//', label='12-0')  # 12-0 bars

    ax.set_xticks(x)
    ax.set_xticklabels(labels, fontsize=12)
    ax.set_ylabel("Mean Monthly Return (%)", fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    ax.legend(loc='upper right', fontsize=12)  # <-- LEGEND ADDED

    plt.tight_layout()
    if save_as:
        plt.savefig(save_as, format='pdf')
    plt.show()

# Call the function
plot_volatility_regime_bar_grouped(
    vol_perf_12_1,
    vol_perf_12_0,
    "Vol_Regime",  # <-- correct column name
    "Mean Monthly Returns by Previous Month Volatility Regime",
    save_as="figure3.pdf"
)



In [ ]:
# Create joint regimes by combining previous month's return and volatility regimes
joint_12_0 = prev_perf_12_0.copy().join(vol_perf_12_0["Vol_Regime"])
joint_12_0["Joint_Regime"] = joint_12_0["PrevMonth_Regime"] + " & " + joint_12_0["Vol_Regime"]

joint_12_1 = prev_perf_12_1.copy().join(vol_perf_12_1["Vol_Regime"])
joint_12_1["Joint_Regime"] = joint_12_1["PrevMonth_Regime"] + " & " + joint_12_1["Vol_Regime"]

# Map joint regimes to clean labels
joint_label_map = {
    "Above Average Previous Return Month & Below Average Volatility": "Stable Trend",
    "Below Average Previous Return Month & Below Average Volatility": "Slow Breakdown",
    "Above Average Previous Return Month & Above Average Volatility": "Overheated",
    "Below Average Previous Return Month & Above Average Volatility": "Stress / Crash"
}

joint_regimes_order = ["Stable Trend", "Slow Breakdown", "Overheated", "Stress / Crash"]

# Compute mean returns for each joint regime and strategy
data_12_1 = [joint_12_1.loc[joint_12_1["Joint_Regime"] == k, "Long_Return"].mean() * 100 
              for k in joint_label_map.keys()]
data_12_0 = [joint_12_0.loc[joint_12_0["Joint_Regime"] == k, "Long_Return"].mean() * 100 
              for k in joint_label_map.keys()]

x = np.arange(len(joint_regimes_order))
bar_width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))  # Slightly smaller width to fit page better

# Plot bars for 12-1 (solid black) and 12-0 (white with hatch)
ax.bar(x - bar_width/2, data_12_1, width=bar_width, color='black', label="12-1")
ax.bar(x + bar_width/2, data_12_0, width=bar_width, color='white', edgecolor='black', hatch='//', label="12-0")

# Set x-axis labels using the clean regime names
ax.set_xticks(x)
ax.set_xticklabels(joint_regimes_order, rotation=0, fontsize=12)

# Axis labels, title, legend, grid
ax.set_ylabel("Mean Monthly Return (%)")
ax.set_title("Mean Monthly Returns by Joint Return–Volatility Regime", fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
ax.grid(axis='y', linestyle='--', alpha=0.7)

# Save figure for LaTeX
plt.tight_layout()
plt.savefig("figure4.pdf")
plt.show()
